<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#import-the-libraries" data-toc-modified-id="import-the-libraries-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>import the libraries</a></span></li><li><span><a href="#Functions-for-discretizing-the-observation-values" data-toc-modified-id="Functions-for-discretizing-the-observation-values-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Functions for discretizing the observation values</a></span></li><li><span><a href="#initialize-the-polecart-environment" data-toc-modified-id="initialize-the-polecart-environment-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>initialize the polecart environment</a></span></li><li><span><a href="#Q-Table-based-algorithm" data-toc-modified-id="Q-Table-based-algorithm-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Q Table based algorithm</a></span></li><li><span><a href="#Q-Network-based-algorithm" data-toc-modified-id="Q-Network-based-algorithm-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Q Network based algorithm</a></span></li></ul></div>

# Deep Reinforcement Learning <a class="tocSkip">

## import the libraries

In [1]:
from collections import deque 

import numpy as np
np.random.seed(123)
print("NumPy:{}".format(np.__version__))

import tensorflow as tf
tf.set_random_seed(123)
print("TensorFlow:{}".format(tf.__version__))

import keras
print("Keras:{}".format(keras.__version__))

import gym
print('OpenAI Gym:',gym.__version__)

NumPy:1.14.5
TensorFlow:1.11.0
Keras:2.2.4
OpenAI Gym: 0.10.8


Using TensorFlow backend.


## Functions for discretizing the observation values

In [2]:
# discretize the value to a state space
def discretize(val,bounds,n_states):
    discrete_val = 0
    if val <= bounds[0]:
        discrete_val = 0
    elif val >= bounds[1]:
        discrete_val = n_states-1
    else:
        discrete_val = int(round( (n_states-1) * 
                                  ((val-bounds[0])/
                                   (bounds[1]-bounds[0])) 
                                ))
    return discrete_val

def discretize_state(vals,s_bounds,n_s):
    discrete_vals = []
    for i in range(len(n_s)):
        discrete_vals.append(discretize(vals[i],s_bounds[i],n_s[i]))
    return np.array(discrete_vals,dtype=np.int)

## initialize the polecart environment

In [3]:
env = gym.make('CartPole-v0')
n_a = env.action_space.n
# number of discrete states for each observation dimension
n_s = np.array([10,10,10,10])   # position, velocity, angle, angular velocity
s_bounds = np.array(list(zip(env.observation_space.low, env.observation_space.high)))
# the velocity and angular velocity bounds are too high so we bound between -1, +1
s_bounds[1] = (-1.0,1.0) 
s_bounds[3] = (-1.0,1.0)   

## Q Table based algorithm

In [4]:
def policy_q_table(state, env):
    # Exploration strategy - Select a random action
    if np.random.random() < explore_rate:
        action = env.action_space.sample()
    # Exploitation strategy - Select the action with the highest q
    else:
        action = np.argmax(q_table[tuple(state)])
    return action

def episode(env, policy, r_max=0, t_max=0):

    # observe initial state
    obs = env.reset()
    state_prev = discretize_state(obs,s_bounds,n_s)
    
    # initialize the variables
    episode_reward = 0
    done = False
    t = 0
    while not done:
        
        # select an action, and observe the next state
        action = policy(state_prev, env)
        obs, reward, done, info = env.step(action)
        state_new = discretize_state(obs,s_bounds,n_s)

        # Update the Q-table 
        best_q = np.amax(q_table[tuple(state_new)])
        bellman_q = reward + discount_rate * best_q
        indices = tuple(np.append(state_prev,action))
        q_table[indices] += learning_rate*( bellman_q - q_table[indices])
    
        # set next state as current state
        state_prev = state_new
        
        episode_reward += reward
        if r_max > 0 and episode_reward > r_max:
            break
        t+=1
        if t_max > 0 and t == t_max:
            break
    return episode_reward
    
    #if return_hist_reward>=episode_reward:
    #    return_val = [np.array(o_list),np.array(a_list),np.array(r_list)]
    #else:
    #    return_val = episode_reward
    #return return_val

# collect observations and rewards for each episode
def experiment(env, policy, n_episodes,r_max=0, t_max=0):
    
    rewards=np.empty(shape=[n_episodes])
    for i in range(n_episodes):
        val = episode(env, policy, r_max, t_max)
        rewards[i]=val
            
    print('Policy:{}, Min reward:{}, Max reward:{}, Average reward:{}'
      .format(policy.__name__,
              np.min(rewards),
              np.max(rewards),
              np.mean(rewards)))


# create a q-table of shape (10,10,10,10, 2) representing S X A -> R
q_table = np.zeros(shape = np.append(n_s,n_a))    
    
learning_rate = 0.8
discount_rate = 0.9
explore_rate = 0.2
n_episodes = 1000

experiment(env, policy_q_table, n_episodes)

Policy:policy_q_table, Min reward:8.0, Max reward:127.0, Average reward:19.434


## Q Network based algorithm

In [5]:
tf.reset_default_graph()
keras.backend.clear_session()

In [7]:
def policy_q_nn(obs, env):
    # Exploration strategy - Select a random action
    if np.random.random() < explore_rate:
        action = env.action_space.sample()
    # Exploitation strategy - Select the action with the highest q
    else:
        action = np.argmax(q_nn.predict(np.array([obs])))
    return action


In [8]:
def episode(env, policy, r_max=0, t_max=0):

    # observe initial state
    obs = env.reset()
    state_prev = discretize_state(obs,s_bounds,n_s)
    
    # initialize the variables
    episode_reward = 0
    done = False
    t = 0
    
    while not done:
        
        action = policy(state_prev, env)
        obs, reward, done, info = env.step(action)
        state_next = discretize_state(obs,s_bounds,n_s)
                                             
        # add the state_prev, action, reward, state_new, done to memory
        memory.append([state_prev,action,reward,state_next,done])
                           
        
        # Generate and update the q_values with 
        # maximum future rewards using bellman function:
        states = np.array([x[0] for x in memory])
        states_next = np.array([np.zeros(4) if x[4] else x[3] for x in memory])
        
        q_values = q_nn.predict(states)
        q_values_next = q_nn.predict(states_next)
        
        for i in range(len(memory)):
            state_prev,action,reward,state_next,done = memory[i]
            if done:
                q_values[i,action] = reward
            else:
                best_q = np.amax(q_values_next[i])
                bellman_q = reward + discount_rate * best_q
                q_values[i,action] = bellman_q
        
        # train the q_nn with states and q_values, same as updating the q_table
        q_nn.fit(states,q_values,epochs=1,batch_size=50,verbose=1)
    
        state_prev = state_next
        
        episode_reward += reward
        if r_max > 0 and episode_reward > r_max:
            break
        t+=1
        if t_max > 0 and t == t_max:
            break
    return episode_reward


In [9]:
# experiment collect observations and rewards for each episode
def experiment(env, policy, n_episodes,r_max=0, t_max=0):
    
    rewards=np.empty(shape=[n_episodes])
    for i in range(n_episodes):
        
        print(i)
        val = episode(env, policy, r_max, t_max)
        #print('episode:{}, reward {}'.format(i,val))
        rewards[i]=val
            
    print('Policy:{}, Min reward:{}, Max reward:{}, Average reward:{}'
        .format(policy.__name__,
              np.min(rewards),
              np.max(rewards),
              np.mean(rewards)))


In [10]:
# create the empty list to contain game memory
memory = deque(maxlen=1000)

# build the Q-Network
from keras.models import Sequential
from keras.layers import Dense
model = Sequential()
model.add(Dense(8,input_dim=4, activation='relu'))
model.add(Dense(2, activation='linear'))
model.compile(loss='mse',optimizer='adam')
model.summary()
q_nn = model
    
learning_rate = 0.8
discount_rate = 0.9
explore_rate = 0.2
n_episodes = 100


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 8)                 40        
_________________________________________________________________
dense_4 (Dense)              (None, 2)                 18        
Total params: 58
Trainable params: 58
Non-trainable params: 0
_________________________________________________________________


In [11]:

experiment(env, policy_q_nn, n_episodes)

0
Epoch 1/1
1/1 [==============================] - 0s 148ms/step - loss: 2.2437
Epoch 1/1
2/2 [==============================] - 0s 1ms/step - loss: 1.2799
Epoch 1/1
3/3 [==============================] - 0s 819us/step - loss: 0.9624
Epoch 1/1
4/4 [==============================] - 0s 601us/step - loss: 3.7918
Epoch 1/1
5/5 [==============================] - 0s 529us/step - loss: 3.1740
Epoch 1/1
6/6 [==============================] - 0s 420us/step - loss: 4.8790
Epoch 1/1
7/7 [==============================] - 0s 387us/step - loss: 4.3096
Epoch 1/1
8/8 [==============================] - 0s 346us/step - loss: 3.8695
Epoch 1/1
9/9 [==============================] - 0s 299us/step - loss: 5.5023
Epoch 1/1
10/10 [==============================] - 0s 252us/step - loss: 5.0142
Epoch 1/1
11/11 [==============================] - 0s 229us/step - loss: 6.6820
Epoch 1/1
12/12 [==============================] - 0s 214us/step - loss: 6.1628
Epoch 1/1
13/13 [==============================] - 0s 186u

104/104 [==============================] - 0s 88us/step - loss: 0.6386
Epoch 1/1
105/105 [==============================] - 0s 90us/step - loss: 0.6233
Epoch 1/1
106/106 [==============================] - 0s 89us/step - loss: 0.6376
10
Epoch 1/1
107/107 [==============================] - 0s 88us/step - loss: 0.6401
Epoch 1/1
108/108 [==============================] - 0s 78us/step - loss: 0.6364
Epoch 1/1
109/109 [==============================] - 0s 78us/step - loss: 0.6253
Epoch 1/1
110/110 [==============================] - 0s 85us/step - loss: 0.6061
Epoch 1/1
111/111 [==============================] - 0s 73us/step - loss: 0.5969
Epoch 1/1
112/112 [==============================] - 0s 75us/step - loss: 0.5936
Epoch 1/1
113/113 [==============================] - 0s 72us/step - loss: 0.5970
Epoch 1/1
114/114 [==============================] - 0s 78us/step - loss: 0.6081
Epoch 1/1
115/115 [==============================] - 0s 73us/step - loss: 0.6099
Epoch 1/1
116/116 [================

205/205 [==============================] - 0s 60us/step - loss: 1.1685
Epoch 1/1
206/206 [==============================] - 0s 64us/step - loss: 1.1633
Epoch 1/1
207/207 [==============================] - 0s 62us/step - loss: 1.1695
Epoch 1/1
208/208 [==============================] - 0s 66us/step - loss: 1.1785
Epoch 1/1
209/209 [==============================] - 0s 61us/step - loss: 1.1691
Epoch 1/1
210/210 [==============================] - 0s 68us/step - loss: 1.1550
Epoch 1/1
211/211 [==============================] - 0s 60us/step - loss: 1.1619
Epoch 1/1
212/212 [==============================] - 0s 62us/step - loss: 1.1628
Epoch 1/1
213/213 [==============================] - 0s 60us/step - loss: 1.1596
Epoch 1/1
214/214 [==============================] - 0s 58us/step - loss: 1.1576
Epoch 1/1
215/215 [==============================] - 0s 66us/step - loss: 1.1574
Epoch 1/1
216/216 [==============================] - 0s 62us/step - loss: 1.1551
Epoch 1/1
217/217 [===================

306/306 [==============================] - 0s 60us/step - loss: 0.9396
Epoch 1/1
307/307 [==============================] - 0s 64us/step - loss: 0.9314
Epoch 1/1
308/308 [==============================] - 0s 60us/step - loss: 0.9350
Epoch 1/1
309/309 [==============================] - 0s 57us/step - loss: 0.9289
Epoch 1/1
310/310 [==============================] - 0s 59us/step - loss: 0.9501
Epoch 1/1
311/311 [==============================] - 0s 59us/step - loss: 0.9552
Epoch 1/1
312/312 [==============================] - 0s 56us/step - loss: 0.9417
Epoch 1/1
313/313 [==============================] - 0s 59us/step - loss: 0.9531
24
Epoch 1/1
314/314 [==============================] - 0s 59us/step - loss: 0.9486
Epoch 1/1
315/315 [==============================] - 0s 55us/step - loss: 0.9416
Epoch 1/1
316/316 [==============================] - 0s 58us/step - loss: 0.9452
Epoch 1/1
317/317 [==============================] - 0s 56us/step - loss: 0.9453
Epoch 1/1
318/318 [================

407/407 [==============================] - 0s 57us/step - loss: 0.7897
30
Epoch 1/1
408/408 [==============================] - 0s 55us/step - loss: 0.7689
Epoch 1/1
409/409 [==============================] - 0s 56us/step - loss: 0.7719
Epoch 1/1
410/410 [==============================] - 0s 56us/step - loss: 0.7748
Epoch 1/1
411/411 [==============================] - 0s 58us/step - loss: 0.7649
Epoch 1/1
412/412 [==============================] - 0s 54us/step - loss: 0.7688
Epoch 1/1
413/413 [==============================] - 0s 56us/step - loss: 0.7588
Epoch 1/1
414/414 [==============================] - 0s 55us/step - loss: 0.7615
Epoch 1/1
415/415 [==============================] - 0s 54us/step - loss: 0.7646
Epoch 1/1
416/416 [==============================] - 0s 56us/step - loss: 0.7589
Epoch 1/1
417/417 [==============================] - 0s 55us/step - loss: 0.7605
Epoch 1/1
418/418 [==============================] - 0s 55us/step - loss: 0.7632
Epoch 1/1
419/419 [================

...

1000/1000 [==============================] - 0s 48us/step - loss: 0.7056
Epoch 1/1
1000/1000 [==============================] - 0s 48us/step - loss: 0.7149
Epoch 1/1
1000/1000 [==============================] - 0s 48us/step - loss: 0.7210
Epoch 1/1
1000/1000 [==============================] - 0s 49us/step - loss: 0.7189
Epoch 1/1
1000/1000 [==============================] - 0s 49us/step - loss: 0.7097
Epoch 1/1
1000/1000 [==============================] - 0s 49us/step - loss: 0.7302
Epoch 1/1
1000/1000 [==============================] - 0s 48us/step - loss: 0.7107
Epoch 1/1
1000/1000 [==============================] - 0s 49us/step - loss: 0.7079
Epoch 1/1
1000/1000 [==============================] - 0s 49us/step - loss: 0.7868
70
Epoch 1/1
1000/1000 [==============================] - 0s 49us/step - loss: 0.7664
Epoch 1/1
1000/1000 [==============================] - 0s 49us/step - loss: 0.7556
Epoch 1/1
1000/1000 [==============================] - 0s 49us/step - loss: 0.7516
Epoch 1/1
1

1000/1000 [==============================] - 0s 48us/step - loss: 0.9826
Epoch 1/1
1000/1000 [==============================] - 0s 50us/step - loss: 0.9642
Epoch 1/1
1000/1000 [==============================] - 0s 49us/step - loss: 0.9590
Epoch 1/1
1000/1000 [==============================] - 0s 48us/step - loss: 0.9488
Epoch 1/1
1000/1000 [==============================] - 0s 49us/step - loss: 0.9335
Epoch 1/1
1000/1000 [==============================] - 0s 48us/step - loss: 0.9742
76
Epoch 1/1
1000/1000 [==============================] - 0s 49us/step - loss: 0.9628
Epoch 1/1
1000/1000 [==============================] - 0s 49us/step - loss: 0.9317
Epoch 1/1
1000/1000 [==============================] - 0s 49us/step - loss: 0.8995
Epoch 1/1
1000/1000 [==============================] - 0s 48us/step - loss: 0.8917
Epoch 1/1
1000/1000 [==============================] - 0s 48us/step - loss: 0.8680
Epoch 1/1
1000/1000 [==============================] - 0s 49us/step - loss: 0.8762
Epoch 1/1
1

1000/1000 [==============================] - 0s 49us/step - loss: 0.9043
Epoch 1/1
1000/1000 [==============================] - 0s 48us/step - loss: 0.9018
Epoch 1/1
1000/1000 [==============================] - 0s 48us/step - loss: 0.9060
Epoch 1/1
1000/1000 [==============================] - 0s 49us/step - loss: 0.8859
Epoch 1/1
1000/1000 [==============================] - 0s 49us/step - loss: 0.8952
Epoch 1/1
1000/1000 [==============================] - 0s 50us/step - loss: 0.9063
Epoch 1/1
1000/1000 [==============================] - 0s 49us/step - loss: 0.9886
84
Epoch 1/1
1000/1000 [==============================] - 0s 48us/step - loss: 0.9903
Epoch 1/1
1000/1000 [==============================] - 0s 48us/step - loss: 0.9857
Epoch 1/1
1000/1000 [==============================] - 0s 48us/step - loss: 0.9936
Epoch 1/1
1000/1000 [==============================] - 0s 48us/step - loss: 0.9728
Epoch 1/1
1000/1000 [==============================] - 0s 49us/step - loss: 0.9765
Epoch 1/1
1

1000/1000 [==============================] - 0s 50us/step - loss: 0.9763
Epoch 1/1
1000/1000 [==============================] - 0s 48us/step - loss: 0.9590
Epoch 1/1
1000/1000 [==============================] - 0s 50us/step - loss: 0.9611
Epoch 1/1
1000/1000 [==============================] - 0s 48us/step - loss: 0.9639
91
Epoch 1/1
1000/1000 [==============================] - 0s 49us/step - loss: 0.9753
Epoch 1/1
1000/1000 [==============================] - 0s 48us/step - loss: 0.9861
Epoch 1/1
1000/1000 [==============================] - 0s 49us/step - loss: 0.9955
Epoch 1/1
1000/1000 [==============================] - 0s 49us/step - loss: 1.0071
Epoch 1/1
1000/1000 [==============================] - 0s 50us/step - loss: 1.0140
Epoch 1/1
1000/1000 [==============================] - 0s 48us/step - loss: 1.0145
Epoch 1/1
1000/1000 [==============================] - 0s 49us/step - loss: 1.0184
Epoch 1/1
1000/1000 [==============================] - 0s 48us/step - loss: 0.9993
Epoch 1/1
1

1000/1000 [==============================] - 0s 49us/step - loss: 1.0209
Epoch 1/1
1000/1000 [==============================] - 0s 49us/step - loss: 1.0193
Epoch 1/1
1000/1000 [==============================] - 0s 49us/step - loss: 1.0007
99
Epoch 1/1
1000/1000 [==============================] - 0s 49us/step - loss: 1.0041
Epoch 1/1
1000/1000 [==============================] - 0s 48us/step - loss: 1.0118
Epoch 1/1
1000/1000 [==============================] - 0s 49us/step - loss: 1.0017
Epoch 1/1
1000/1000 [==============================] - 0s 49us/step - loss: 1.0137
Epoch 1/1
1000/1000 [==============================] - 0s 48us/step - loss: 0.9771
Epoch 1/1
1000/1000 [==============================] - 0s 48us/step - loss: 0.9865
Epoch 1/1
1000/1000 [==============================] - 0s 48us/step - loss: 1.0016
Epoch 1/1
1000/1000 [==============================] - 0s 49us/step - loss: 1.0139
Epoch 1/1
1000/1000 [==============================] - 0s 49us/step - loss: 1.0332
Policy:poli